# Clean

Clean up and remove unnecessary Anilist data

In [121]:
import csv
import json
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime

data_dir = os.path.abspath('data')
# today = datetime.today().strftime('%Y%m%d')
today = datetime.strptime('2022-09-27', '%Y-%m-%d').strftime('%Y%m%d')

anime_df = pd.read_csv(os.path.join(data_dir, f'anime-{today}-raw.csv'), parse_dates=[8,9])
user_df = pd.read_csv(os.path.join(data_dir, f'user-{today}-raw.csv'), parse_dates=[4])

## Check Missing Data

In [122]:
def get_missing(df: DataFrame) -> DataFrame:
    df = DataFrame(data=df.isnull().mean().round(4).mul(100).sort_values(ascending=False), columns=['percent_missing'])
    return df[df['percent_missing'] > 0]

def get_missing_user(df: DataFrame) -> DataFrame:
    return get_missing(df[df['status'].isin(['COMPLETED','CURRENT'])])

def get_missing_anime(df: DataFrame) -> DataFrame:
    df['genres'] = df['genres'].mask(df['genres'] == '[]', None)
    df['tags'] = df['tags'].mask(df['tags'] == '[]', None)
    df['studios'] = df['studios'].mask(df['studios'] == '[]', None)
    return get_missing(df[df['status'].isin(['FINISHED','RELEASING'])])

print(f"Missing User Data:\n{get_missing_user(user_df)}\n")
print(f"Missing Anime Data:\n{get_missing_anime(anime_df)}")

Missing User Data:
             percent_missing
completedAt            15.22

Missing Anime Data:
                     percent_missing
next_airing_episode            99.34
title_english                  52.40
studios                        46.67
season                         32.79
season_year                    32.79
season_int                     31.50
average_score                  29.68
tags                           23.68
source                         14.25
genres                         13.92
description                     5.87
end_date                        2.06
title_native                    1.47
duration_mins                   1.28
episodes                        0.74
mean_score                      0.15
start_date                      0.03
format                          0.01


## Clean User Data

In [123]:
def clean_user_data(df: DataFrame) -> DataFrame:
    return df.drop('completedAt', axis=1, errors='ignore')

user_df = clean_user_data(user_df)

## Clean Anime Data

In [124]:
def fill_season(dt: datetime) -> str:
    if dt.month >= 9 and dt.month <= 11:
        return 'FALL'
    elif dt.month >= 6 and dt.month <= 8:
        return 'SUMMER'
    elif dt.month >= 3 and dt.month <= 5:
        return 'SPRING'
    else:
        return 'WINTER'
    
def fill_season_year(dt: datetime) -> int:
    return 

def extract_names(s: str) -> str:
    return json.dumps([s['name'] for s in json.loads(s)]) if s else None

def test_df(x):
    print(type(x))

def clean_anime_data(df: DataFrame) -> DataFrame:
    # drop unusable columns
    df.drop([
        'title_english', 'title_romaji', 'title_native', 'type', 'description',
        'popularity', 'mean_score', 'average_score', 'season_int', 'end_date'
    ], axis=1, errors='ignore', inplace=True)

    # drop unusable rows
    # df.drop(df[~df['status'].isin(['FINISHED','RELEASING'])].index, axis=0, inplace=True)
    # df.drop(df[~df['format'].isin(['TV','MOVIE','SPECIAL','ONA','OVA'])].index, axis=0, inplace=True)
    # df.dropna(subset=['duration_mins'], inplace=True)

    # fix remaining data
    # df['season'] = df.filter(['season','start_date'])[df['season'].isnull()].apply(lambda x: fill_season(x['start_date']), axis=1)
    # df['season_year'] = df.filter(['season','start_date'])[df['season'].isnull()].apply(lambda x: x['start_date'].year, axis=1)
    # df['studios'] = df[~df['studios'].isnull()].apply(lambda x: extract_names(x['studios']), axis=1)
    # df['tags'] = df[~df['tags'].isnull()].apply(lambda x: extract_names(x['tags']), axis=1)

    df['source'] = df['source'].fillna('OTHER')
    df['season'] = df['season'].fillna(df['start_date'].apply(fill_season))
    df['season_year'] = df['season_year'].fillna(df['start_date'].apply(lambda x: x.year if x else None))
    df['studios'] = df['studios'].apply(extract_names)
    df['tags'] = df['tags'].apply(extract_names)
    
    # df['studios'] = df['studios'].fillna(df['studios'].apply(lambda x: extract_names(x['studios'])))
    # print(df.head(3))

    # TODO: df['season'] = fill_season(df['start_date'])
    # TODO: df['season_year'] = df['start_date'].year
    # TODO: df['episodes'] = df['next_airing_episodes']

    # drop definitiely unusable rows
    df.dropna(subset=['episodes'], inplace=True)

    # drop vestigial data
    df.drop(['next_airing_episode'], axis=1, errors='ignore', inplace=True)
    #'start_date'

    return df

anime_df = clean_anime_data(anime_df)

## Verify Missing Data Resolved

In [125]:
print(f"Missing User Data:\n{get_missing_user(user_df)}\n")
print(f"Missing Anime Data:\n{get_missing_anime(anime_df)}")

Missing User Data:
Empty DataFrame
Columns: [percent_missing]
Index: []

Missing Anime Data:
               percent_missing
studios                  46.71
tags                     23.68
genres                   13.97
duration_mins             1.12
start_date                0.03
season_year               0.03


# Combine Anime and User Data

In [126]:
combined_df = pd.merge(user_df[user_df['status'].isin(['CURRENT','COMPLETED'])], anime_df, left_on='media_id', right_on='id', how='inner')
print(f"Missing Data (before):\n{get_missing(combined_df)}\n")

combined_df.drop(combined_df[combined_df['score'] == 0].index, axis=0, inplace=True)
combined_df.dropna(subset=['genres'], inplace=True)
combined_df.dropna(subset=['studios'], inplace=True)

print(f"Missing Data (after):\n{get_missing(combined_df)}\n")
combined_df.to_csv(os.path.join(data_dir, f'user-{today}-enriched.csv'))

print(user_df.shape[0] - combined_df.shape[0], 'rows dropped')
print(f'{combined_df.shape[0]}', 'rows written')

Missing Data (before):
               percent_missing
studios                   3.64
duration_mins             0.33
genres                    0.17

Missing Data (after):
Empty DataFrame
Columns: [percent_missing]
Index: []

193 rows dropped
567 rows written
